<a href="https://colab.research.google.com/github/Richardjmorton/KL7002_students/blob/main/Labs1_Reading_in_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
try:
  import sunpy
  print("module 'sunpy' is installed")
except ModuleNotFoundError:
  !pip install sunpy



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 45.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 56.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 29.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 18.2 MB/s eta 0:00:00


In [3]:
#module imports
import numpy as np
import matplotlib.pyplot as plt

# makes plots appear in notebook
%matplotlib inline

from IPython.display import Image

from astropy import units as u

from pathlib import Path

## Paths

When working with data, it will be stored in some location in your computer. You may launch Python (or a notebook) from a different location and require access to the data files stored elsewhere.

It can be helpful to use Python's native module *os* which permits you to use your [operating systems functionality](https://docs.python.org/3.7/library/os.html).

When you run Python from the terminal, it starts in the directory or folder you are currently working in (unsurprisingly called the *current working directory*). If you use a developer environment, e.g. Pycharm, then you can choose a default working directory.

If you don't know what directory you are working in, you can query this:

In [4]:
# When you run this you will get a different answer...which is *your* CWD

import os
print(os.getcwd()) # returns current working directory

/content


You can see that the result is a path to the directory on my computer where I am working.

The first part of the output refers to the home directory, `/Users/richardmorton`. To find your home directory, you can use the following function in *os*

In [5]:
# When you run this you will get a different answer

from os.path import expanduser

home = expanduser("~")

print(home)

/root


You can build paths up in order to access different directories and files.

As as simple example, suppose your working in a directory called `top_deck` and your data is stored in a sub folder, called `data_folder`, e.g.

```
top_deck/
   |- analysis_notebook.ipynb
   |- data_folder
      |- star_data.fits
      |- table.csv
```

You can build paths by using strings in the following way:

In [6]:
# create string for file path
path = (home, 'top_deck', 'data_folder/table.csv')

s = '/'
filepath = s.join(path)
print(filepath)

/root/top_deck/data_folder/table.csv


These strings can then be used with functions that read in information within files.

<br>

When writing code for others, the above way of constructing paths can be system dependent, i.e., depends on if you are using Windows, Mac OsX etc.In the above example, the `/` used for separating directories is the Mac OsX way, while windows uses `\`.

There are of course options to build system agnostic paths. This can be done using the `pathlib` module. You can find a detailed example of how the is module works [here](https://realpython.com/python-pathlib/).



In [7]:
# import Path class from pathlib
from pathlib import Path

#
Path.home()


PosixPath('/root')

We can see here a `PosixPath` object is created, this would have been a `WindowsPath` object if I had run this code on windows.

There are a number of useful methods that come with the `Path` class that make it easier to build and inspect paths.

For example, paths can be joined using the `/` operator.

In [8]:
root = Path.home()
new_path = root/ 'top_deck' / 'data_folder'
new_path

PosixPath('/root/top_deck/data_folder')

Here I just show a few other useful methods

In [9]:
# Checking to see if a file or directory exists:
new_path.exists()

False

In [10]:
# Access the name of the file that the path is pointing to
path_to_data = new_path / "table.csv"
path_to_data.name

'table.csv'

In [12]:
# Create a new directory, named 'new_folder' within the root directory
new_dir = root / "new_folder"
new_dir.mkdir()

FileExistsError: ignored